05-xrefs
Get xrefs from a variety of sources
- Drugs: 
UMLS has mesh xrefs. From mesh, we can get UNII and CAS. From UNII_FDA, we can get inchikeys 
(lookup using cas or unii). From chembl, we can get chembl IDs from the inchikeys
So: UMLS -> mesh -> unii/cas -> inchikey -> chembl
insane, I know.
- Anatomy: uberon has umls xrefs
- disease: DO has umls, umls has NCI, ICD10PCS, SNOMEDCT_US, ICD10CM, OMIM
- proteins: umls has uniprot xrefs
- biological_process_or_activity/activity_and_behavior: umls has GO
- gene: umls has HGNC and OMIM

In [1]:
import sys
import os
import pickle
%matplotlib inline
import pandas as pd
from numpy import nan
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from itertools import chain
from more_itertools import chunked
from collections import Counter
from pprint import pprint
import requests
from pyquery import PyQuery as pq
from wikidataintegrator import wdi_helpers, wdi_core, wdi_login

In [2]:
DATA  = 'data/'

UMLS  = DATA+"2020AA-full/2020AA/META/"
MRCONSO_ENG_ARCHIVE = UMLS+"MRCONSO_ENG.RRF.gz"
MRSAT_ARCHIVE = UMLS+"MRSAT.RRF.gz"

EDGES_BIOLINK_CSV = DATA+"edges_biolink.csv"
NODES_BIOLINK_CSV = DATA+"nodes_biolink.csv"

UNII_RECORDS = DATA+"UNII_Records_23May2020.txt"

XREFS_SHELVE = DATA+"xrefs.shelve"

UBERON_CSV = DATA+"uberon.csv"
DOID_CSV = DATA+"doid.csv"

MESH_XREFS_CSV = DATA+"mesh_xrefs.csv"
NODES_XREF_CSV = DATA+"nodes_xref.csv"

In [3]:
uri_to_curie = lambda s: s.split("/")[-1].replace("_", ":")

In [4]:
nodes = pd.read_csv(NODES_BIOLINK_CSV, index_col=0)

In [5]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_category
ID,,,,
C3842672,Day 7,T033,Finding,disease_or_phenotypic_feature
C0020684,Hypoxanthine,T123|T109,Biologically Active Substance|Organic Chemical,chemical_substance
C0853225,INR Increased,T033,Finding,disease_or_phenotypic_feature
C1513022,Mature Centriole,T026,Cell Component,cell_component
C0267183,Hourglass stricture or stenosis of stomach,T047,Disease or Syndrome,disease_or_phenotypic_feature


In [6]:
nodes.blm_category.value_counts()

chemical_substance                57087
protein                           27274
disease_or_phenotypic_feature     26634
gene                              23704
gross_anatomical_structure         8211
biological_process_or_activity     7260
anatomical_entity                  2275
cell_component                     1673
cell                               1193
activity_and_behavior               845
phenotypic_feature                  252
genomic_entity                      170
Name: blm_category, dtype: int64

In [7]:
## parse UMLS flat file to get all UMLS xrefs
# see: https://www.ncbi.nlm.nih.gov/books/NBK9685/

In [8]:
names = "CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X".split(",")
umls = pd.read_csv(MRCONSO_ENG_ARCHIVE, delimiter="|", names=names, index_col=None)
# only get CUIs in our list of nodes
umls = umls[umls.CUI.isin(nodes.index)]

/Users/richard/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
umls['xref'] = umls.SAB + ":" + umls.CODE.map(str)
# fix this MSH MESH nonsense
umls.xref = umls.xref.str.replace("MSH:", "MESH:")
# NCI_FDA is UNII
umls.xref = umls.xref.str.replace("NCI_FDA:", "UNII:")

In [10]:
umls.head(2)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X,xref
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,RXNORM:1926948
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,MTH:NOCODE


In [11]:
XREF = dict(umls.groupby("CUI")['xref'].apply(set))
XREF = defaultdict(set, XREF)
print(XREF['C0591520'])

{'CHV:0000041179', 'MESH:D000068298'}


In [12]:
# all xrefs we get from umls
# Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in XREF.values()])))

### Chemicals and drugs

In [13]:
# what xrefs are on chemicals?
chem_umls = nodes[nodes.blm_category == "chemical_substance"].index
xref_chem = {k:v for k,v in XREF.items() if k in chem_umls}
print(len(chem_umls))
c = Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in xref_chem.values()])))
pprint(c.most_common(25))
# nearly all have a mesh ID. not much of anything else
# neither mesh nor umls have inchikeys, or inchi, or smiles or anything usefull for linking out
# blech

57087
[('MESH', 51116),
 ('NCI', 11037),
 ('RXNORM', 9120),
 ('CHV', 8325),
 ('MTH', 7286),
 ('MTHSPL', 6650),
 ('UNII', 6567),
 ('LNC', 5556),
 ('VANDF', 4038),
 ('DRUGBANK', 3496),
 ('ATC', 3204),
 ('PDQ', 2963),
 ('CSP', 2896),
 ('AOD', 1945),
 ('USP', 1884),
 ('LCH_NW', 1825),
 ('NCI_NCI-GLOSS', 1506),
 ('LCH', 1173),
 ('NCI_CTRP', 1007),
 ('USPMG', 938),
 ('NCI_DTP', 834),
 ('NCI_DCP', 734),
 ('NCI_CRCH', 235),
 ('HL7V3.0', 167),
 ('SPN', 137)]


In [14]:
pd.set_option("display.width", 120)

URL = "http://id.nlm.nih.gov/mesh/sparql"
PREFIX = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
"""

def sparql_query(query):
    params = {'query': PREFIX + query, 'format': 'JSON', 'limit': 1000, 'offset': 0}
    r = requests.get(URL, params=params)
    res = [{k: v['value'] for k, v in x.items()} for x in r.json()['results']['bindings']]
    t = tqdm()
    while True:
        t.update(1)
        params['offset'] += 1000
        r = requests.get(URL, params=params).json()['results']['bindings']
        if not r:
            break
        res.extend([{k: v['value'] for k, v in x.items()} for x in r])
    df = pd.DataFrame(res)
    return df

In [15]:
query = """
SELECT distinct ?mesh ?meshLabel ?r ?rr
FROM <http://id.nlm.nih.gov/mesh> WHERE {
  ?mesh meshv:active 1 .
  ?mesh meshv:preferredMappedTo ?p .
  ?p meshv:treeNumber ?treeNum .
  FILTER(STRSTARTS(STR(?treeNum), "http://id.nlm.nih.gov/mesh/D")) .
  ?mesh rdfs:label ?meshLabel .
  ?mesh meshv:preferredConcept [meshv:registryNumber ?r] .
  #OPTIONAL {?mesh meshv:preferredConcept [meshv:relatedRegistryNumber ?rr]}
}
"""
df = sparql_query(query)

218it [08:23,  2.78s/it]

In [16]:
df.r = df.r.replace("0", nan)
df.dropna(subset=["r"], inplace=True)
df = df[~df.r.str.startswith("EC ")]
df.mesh = df.mesh.str.replace("http://id.nlm.nih.gov/mesh/", "")
df.set_index("mesh", inplace=True)

In [17]:
df.to_csv(MESH_XREFS_CSV)
df.head()

,meshLabel,r
mesh,,
C000629587,boron tribromide,A453DV9339
C016271,bromine chloride,7G62XY5724
C016769,Syn-ergel,66799-40-4
C017138,CMB-dextran,37307-31-6
C017827,bacterio-opsin,54577-62-7


In [18]:
mesh_xrefs = pd.read_csv(MESH_XREFS_CSV, index_col=0)
mesh_xrefs.r = mesh_xrefs.r.apply(lambda x: "CAS:" + x if "-" in x else "UNII:" + x)
mesh_xrefs = mesh_xrefs.groupby("mesh").r.apply(set).to_dict()
mesh_xrefs = {"MESH:"+k:v for k,v in mesh_xrefs.items()}
len(mesh_xrefs)

47623

In [19]:
for k,v in xref_chem.items():
    for vv in list(v):
        if vv in mesh_xrefs:
            v.update(mesh_xrefs[vv])

In [20]:
# download: 'http://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip'
unii_df = pd.read_csv(UNII_RECORDS, dtype=str, sep='\t', low_memory=False)
unii_df.dropna(subset=['INCHIKEY'], inplace=True)

In [21]:
unii_df.head()

,UNII,PT,RN,EC,NCIT,RXCUI,PUBCHEM,ITIS,NCBI,PLANTS,GRIN,MPNS,INN_ID,MF,INCHIKEY,SMILES,INGREDIENT_TYPE
0,00174624E2,CHF-6333 CATION,1613620-10-2,NaN,NaN,NaN,76285164,NaN,NaN,NaN,NaN,NaN,NaN,C27H28F3N6O3,IHTRPSMRGYWUIM-HSZRJFAPSA-O,COC(=O)C1=C(C)N(C2=NNC(=O)N2[C@@H]1C3=CC=C(C=C...,IONIC MOIETY
1,0129526470,"5,8-DIMETHOXY(1,2,4)TRIAZOLO(1,5-C)PYRIMIDIN-2...",219715-62-5,NaN,NaN,NaN,11446888,NaN,NaN,NaN,NaN,NaN,NaN,C7H9N5O2,DBJPBHJHAPAUQU-UHFFFAOYSA-N,COC1=CN=C(OC)N2N=C(N)N=C12,INGREDIENT SUBSTANCE
3,0377415922,"N-DESMETHYLVENLAFAXINE, (S)-",392332-59-1,NaN,NaN,NaN,9860056,NaN,NaN,NaN,NaN,NaN,NaN,C16H25NO2,MKAFOJAJJMUXLW-OAHLLOKOSA-N,CNC[C@H](C1=CC=C(OC)C=C1)C2(O)CCCCC2,INGREDIENT SUBSTANCE
5,0480546720,HOMOCYCLOLEUCINE HYDROCHLORIDE,39692-17-6,254-594-3,NaN,NaN,2724466,NaN,NaN,NaN,NaN,NaN,NaN,C7H13NO2.ClH,GTKXSYHXQSKWNP-UHFFFAOYSA-N,Cl.NC1(CCCCC1)C(O)=O,INGREDIENT SUBSTANCE
6,0503177591,SUCCINALDEHYDE,638-37-9,211-333-8,NaN,NaN,12524,NaN,NaN,NaN,NaN,NaN,NaN,C4H6O2,PCSMJKASWLYICJ-UHFFFAOYSA-N,O=CCCC=O,INGREDIENT SUBSTANCE


In [22]:
n=0
for k,v in tqdm_notebook(xref_chem.items()):
    for vv in list(v):
        if vv.startswith("UNII:"):
            xref = vv.replace("UNII:", "")
            s = unii_df.query("UNII == @xref").INCHIKEY
            if not s.empty:
                n+=1
                v.add("INCHIKEY:" + list(s)[0])

In [23]:
xref_inchi = {k:v for k,v in xref_chem.items() if any(vv.startswith("INCHIKEY:") for vv in v)}
xref_inchi = {k:[vv for vv in v if vv.startswith("INCHIKEY:")][0].replace("INCHIKEY:", "") for k,v in xref_inchi.items()}
print(len(xref_inchi))
list(xref_inchi.items())[:4]

11269


[('C0000248', 'CZIHNRWJTSTCEX-UHFFFAOYSA-N'),
 ('C0000294', 'XOGTZOOQQBDUSI-UHFFFAOYSA-M'),
 ('C0000378', 'QXWYKJLNLSIPIN-JGVFFNPUSA-N'),
 ('C0000399', 'CBOKZNLSFMZJJA-PEBGCTIMSA-N')]

In [24]:
url = "https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_structures__standard_inchi_key__in={}&format=json&limit=100"
for chunk in tqdm(chunked(xref_inchi.items(), 100), total=len(xref_inchi)/100):
    chunk = dict(chunk)
    chunk = {v:k for k,v in chunk.items()}
    inchis = ",".join(chunk)
    mols = requests.get(url.format(inchis)).json()['molecules']
    for m in mols:
        chembl = m['molecule_chembl_id']
        inchi = m['molecule_structures']['standard_inchi_key']
        XREF[chunk[inchi]].add("CHEMBL:" + chembl)


  0%|          | 0/112.69 [00:00<?, ?it/s]
  1%|          | 1/112.69 [00:04<09:15,  4.98s/it]
  2%|▏         | 2/112.69 [00:09<08:40,  4.70s/it]
  3%|▎         | 3/112.69 [00:13<08:32,  4.68s/it]
  4%|▎         | 4/112.69 [00:18<08:22,  4.62s/it]
  4%|▍         | 5/112.69 [00:22<08:17,  4.62s/it]
  5%|▌         | 6/112.69 [00:26<07:51,  4.42s/it]
  6%|▌         | 7/112.69 [00:30<07:21,  4.17s/it]
  7%|▋         | 8/112.69 [00:33<06:49,  3.92s/it]
  8%|▊         | 9/112.69 [00:37<06:37,  3.83s/it]
  9%|▉         | 10/112.69 [00:44<08:17,  4.85s/it]
 10%|▉         | 11/112.69 [00:51<09:24,  5.55s/it]
 11%|█         | 12/112.69 [00:56<09:07,  5.43s/it]
 12%|█▏        | 13/112.69 [01:00<08:22,  5.04s/it]
 12%|█▏        | 14/112.69 [01:05<08:01,  4.88s/it]
 13%|█▎        | 15/112.69 [01:09<07:29,  4.60s/it]
 14%|█▍        | 16/112.69 [01:13<06:56,  4.31s/it]
 15%|█▌        | 17/112.69 [01:18<07:17,  4.57s/it]
 16%|█▌        | 18/112.69 [01:22<07:00,  4.44s/it]
 17%|█▋        | 19/112.69 [0

In [25]:
len({k:v for k,v in XREF.items() if any(vv.startswith("CHEMBL:") for vv in v)})

8736

In [26]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

In [27]:
## UBERON, uses uberon.csv generated from the uberon.owl file previously downloaded (see README)

In [28]:
df = pd.read_csv(UBERON_CSV)
df = df[df.xref.str.startswith("UMLS:")]
df.xref = df.xref.str.replace("UMLS:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
9755,UBERON:0007227,C0228757
11788,UBERON:0002965,C0175263
11853,UBERON:0001532,C0007276
11897,UBERON:0014636,C0228627
11904,UBERON:0002610,C0152411


In [29]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [30]:
XREF['C1272528']

{'MTH:NOCODE', 'UBERON:0006472'}

In [31]:
## DOID, uses doid.csv generated from the doi.owl file previously downloaded (see README)

In [32]:
df = pd.read_csv(DOID_CSV)
df.dropna(inplace=True)
df = df[df.xref.str.startswith("UMLS_CUI:")]
df.xref = df.xref.str.replace("UMLS_CUI:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
6,DOID:3864,C1332188
7,DOID:3864,C0278876
21,DOID:699,C0162670
27,DOID:0111831,C1844936
34,DOID:0111618,C1853116


In [33]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [34]:
XREF['C0263518']

{'DOID:1943',
 'HPO:HP:0025470',
 'ICD9CM:704.02',
 'MTH:NOCODE',
 'NCI:C112200',
 'NCI_NICHD:C112200'}

In [35]:
XREF['C0591520']

{'CHV:0000041179', 'MESH:D000068298'}

## proteins

In [36]:
# See README regarding MRSAT_ARCHIVE
names = list("abcdefghijklmn")
iter_csv = pd.read_csv(MRSAT_ARCHIVE, delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_uniprot = dict()
for chunk in tqdm(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "SWISS_PROT"]
    d = dict(zip(chunk.a, chunk.k))
    umls_uniprot.update(d)


  0%|          | 0/67.668372 [00:00<?, ?it/s]
  1%|▏         | 1/67.668372 [00:03<03:33,  3.20s/it]
  3%|▎         | 2/67.668372 [00:06<03:25,  3.14s/it]
  4%|▍         | 3/67.668372 [00:09<03:22,  3.13s/it]
  6%|▌         | 4/67.668372 [00:12<03:14,  3.06s/it]
  7%|▋         | 5/67.668372 [00:15<03:09,  3.03s/it]
  9%|▉         | 6/67.668372 [00:18<03:06,  3.02s/it]
 10%|█         | 7/67.668372 [00:20<02:59,  2.96s/it]
 12%|█▏        | 8/67.668372 [00:23<02:54,  2.93s/it]
 13%|█▎        | 9/67.668372 [00:26<02:51,  2.91s/it]
 15%|█▍        | 10/67.668372 [00:29<02:44,  2.85s/it]
 16%|█▋        | 11/67.668372 [00:32<02:41,  2.85s/it]
 18%|█▊        | 12/67.668372 [00:34<02:35,  2.79s/it]
 19%|█▉        | 13/67.668372 [00:37<02:32,  2.79s/it]
 21%|██        | 14/67.668372 [00:40<02:27,  2.75s/it]
 22%|██▏       | 15/67.668372 [00:43<02:24,  2.75s/it]
 24%|██▎       | 16/67.668372 [00:45<02:22,  2.75s/it]
 25%|██▌       | 17/67.668372 [00:48<02:21,  2.79s/it]/Users/richard/opt/anaconda3

In [37]:
len(umls_uniprot)

4916

In [38]:
for umls, uniprot in umls_uniprot.items():
    XREF[umls].add("UNIPROT:" + uniprot)

In [39]:
XREF['C0215993']

{'MESH:C081092',
 'MTH:NOCODE',
 'NCI:C127008',
 'NCI_CTRP:C127008',
 'UNIPROT:Q04756'}

In [40]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

In [41]:
nodes['xrefs'] = nodes.index.map(lambda x: ";".join(XREF.get(x,list())))

In [42]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_category,xrefs
ID,,,,,
C3842672,Day 7,T033,Finding,disease_or_phenotypic_feature,LNC:LA16590-4
C0020684,Hypoxanthine,T123|T109,Biologically Active Substance|Organic Chemical,chemical_substance,LNC:LP15651-0;NCI:C29105;CHV:0000006506;MESH:D...
C0853225,INR Increased,T033,Finding,disease_or_phenotypic_feature,OMIM:MTHU055758;NCI_CTCAE:E12175;NCI:C78366;CH...
C1513022,Mature Centriole,T026,Cell Component,cell_component,NCI:C33060
C0267183,Hourglass stricture or stenosis of stomach,T047,Disease or Syndrome,disease_or_phenotypic_feature,MTHICD9:537.6;ICD9CM:537.6;DOID:12234


In [43]:
nodes.to_csv(NODES_XREF_CSV)